# 计算测试集图像语义特征

抽取Pytorch训练得到的图像分类模型中间层的输出特征，作为输入图像的语义特征。

计算测试集所有图像的语义特征，使用t-SNE和UMAP两种降维方法降维至二维和三维，可视化。

分析不同类别的语义距离、异常数据、细粒度分类、高维数据结构。

同济子豪兄：https://space.bilibili.com/1900783

[代码运行云GPU环境](https://featurize.cn/?s=d7ce99f842414bfcaea5662a97581bd1)：GPU RTX 3060、CUDA v11.2

## 导入工具包

In [28]:
from tqdm import tqdm

import pandas as pd
import numpy as np

import torch

import cv2
from PIL import Image

# 忽略烦人的红色提示
import warnings
warnings.filterwarnings("ignore")

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 图像预处理

In [29]:
from torchvision import transforms

# # 训练集图像预处理：缩放裁剪、图像增强、转 Tensor、归一化
# train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
#                                       transforms.RandomHorizontalFlip(),
#                                       transforms.ToTensor(),
#                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#                                      ])

# 测试集图像预处理-RCTN：缩放、裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

## 导入训练好的模型

In [30]:
model = torch.load( r'E:\MV-Code-202018010103-Lucy\main\Train_Custom_Dataset\图像分类\3-【Pytorch】迁移学习训练自己的图像分类模型\checkpoint\best-1.000.pth')
model = model.eval().to(device)

## 抽取模型中间层输出结果作为语义特征

In [31]:
from torchvision.models.feature_extraction import create_feature_extractor

In [32]:
model_trunc = create_feature_extractor(model, return_nodes={'avgpool': 'semantic_feature'})

## 计算单张图像的语义特征

In [33]:
img_path = r"D:\dataset\sr\train\parasitized\C39P4thinF_original_IMG_20150622_105554_cell_9_RCAN_BIX4-official.png"
img_pil = Image.open(img_path)
input_img = test_transform(img_pil) # 预处理
input_img = input_img.unsqueeze(0).to(device)
# 执行前向预测，得到指定中间层的输出
pred_logits = model_trunc(input_img) 

In [34]:
pred_logits['semantic_feature'].squeeze().detach().cpu().numpy().shape

(912,)

In [35]:
# pred_logits['semantic_feature'].squeeze().detach().cpu().numpy()

## 载入测试集图像分类结果

In [36]:
df = pd.read_csv('测试集预测结果.csv')

In [37]:
df.head()

,图像路径,标注类别ID,标注类别名称,top-1-预测ID,top-1-预测名称,top-2-预测ID,top-2-预测名称,top-n预测正确,parasitized-预测置信度,uninfected-预测置信度
0,D:\dataset\sr\val\parasitized\C100P61ThinF_IMG...,0,parasitized,0,parasitized,1,uninfected,True,0.999998,2.221901e-06
1,D:\dataset\sr\val\parasitized\C100P61ThinF_IMG...,0,parasitized,0,parasitized,1,uninfected,True,0.999980,2.029260e-05
2,D:\dataset\sr\val\parasitized\C100P61ThinF_IMG...,0,parasitized,0,parasitized,1,uninfected,True,1.000000,3.772365e-10
3,D:\dataset\sr\val\parasitized\C100P61ThinF_IMG...,0,parasitized,0,parasitized,1,uninfected,True,1.000000,2.734464e-08
4,D:\dataset\sr\val\parasitized\C100P61ThinF_IMG...,0,parasitized,0,parasitized,1,uninfected,True,1.000000,2.457842e-09


## 计算测试集每张图像的语义特征

In [38]:
encoding_array = []
img_path_list = []

for img_path in tqdm(df['图像路径']):
    img_path_list.append(img_path)
    img_pil = Image.open(img_path).convert('RGB')
    input_img = test_transform(img_pil).unsqueeze(0).to(device) # 预处理
    feature = model_trunc(input_img)['semantic_feature'].squeeze().detach().cpu().numpy() # 执行前向预测，得到 avgpool 层输出的语义特征
    encoding_array.append(feature)
encoding_array = np.array(encoding_array)

100%|██████████| 5512/5512 [01:53<00:00, 48.49it/s]


In [39]:
encoding_array.shape

(5512, 912)

## 保存为本地的.npy文件

In [40]:
# 保存为本地的 npy 文件
np.save('测试集语义特征.npy', encoding_array)